# Quantum Teleporation Application

In this demo, we'll demonstrate the simulation of the teleportation application. The network topology will have two nodes: Alice and Bob. Alice will teleport a qubit to Bob. 

## 1. The Basics

The figure below shows the quantum circuit of teleportation, as well as showing which qubits belong to Alice and which qubit belongs to Bob.

<img src="../figures/teleport-circuit.png" width="600">

**Set up**: Assume node Alice and Bob both have two quantum memories, one labeled as data memory and the other labeled as communication memory. There is a quantum channel between Alice and Bob, and the BSM node is in the middle of the quantum channel.

Let's break down quantum teleportation into 4 steps.

**Step 1**: The tradition is to let Alice send a qubit to Bob. So, Alice prepares $\ket{\psi}$ at memory $d_1$.

<img src="../figures/teleport-1.png" width="600">

**Step 2**: Generate an entangled pair between Alice's memory $c_1$ and Bob's memory $c_2$.

<img src="../figures/teleport-2.png" width="600">

**Step 3**: Alice does a Bell measurement on it's memory $d_1$ and $c_1$. The measurement results are sent to Bob.

<img src="../figures/teleport-3.png" width="600">

**Step 4**: Bob does the correction on $c_2$ upon receiving the measurement results. Then $\ket{\psi}$ is teleported to Bob's memory $c_2$.

<img src="../figures/teleport-4.png" width="600">

## 2. Show Me The Code

**The imports**

- `log`: defines the behavior for the default SeQUeNCe logging system
- `RequestApp`: is an example application included with SeQUeNCe. We will investigate its behavior through a class that inherits from this
- `DQCNode`: Node that supports Distributed Quantum Computing
- `TeleportMsgType`: Different types of teleportation messages.
- `TeleportProtocol`: A network protocol that does the teleportation
- `TeleportMessage`: Classical message used to convey the Pauli corrections (x, z) from sender to receiver during teleportation.
- `DQCNetTopo`: Class for generating a distributed quantum computing network with distributed quantum computing nodes
- `MemoryInfo`: Class to track memory information parameters for memory manager.
- `random_state`: Generate a random quantum state
- `verify_same_state_vector`: verify whether two state vectors are the same
- `Process`: Defines a process, which is performed when an event is executed.
- `Event`: Class of events for simulation

In [1]:
import numpy as np
import sequence.utils.log as log
from sequence.app.request_app import RequestApp
from sequence.topology.node import DQCNode
from sequence.entanglement_management.teleportation import TeleportMsgType, TeleportProtocol, TeleportMessage
from sequence.topology.dqc_net_topo import DQCNetTopo
from sequence.resource_management.memory_manager import MemoryInfo
from sequence.kernel.quantum_utils import random_state, verify_same_state_vector
from sequence.constants import MILLISECOND
from sequence.kernel.process import Process
from sequence.kernel.event import Event

### 2.1 The Teleportation App

The class `TeleportApp` is responsible for managing quantum teleportation between quantum nodes. It utilizes the `TeleportProtocol` to handle the teleportation process, including the reservation of entangled pairs and the application of corrections based on classical messages.

In [2]:
class TeleportApp(RequestApp):
    """Code for the teleport application.

    TeleportApp is a specialized RequestApp that implements quantum teleportation.
    It handles the teleportation protocol between two quantum nodes (Alice and Bob).

    Attributes:
        node (DQCNode): The quantum node this app is attached to.
        name (str): The name of the teleport application.
        results (list): A list of results of (timestamp, teleported_state)
        teleport_protocols (list[TeleportProtocol]): A list of teleportation protocol instances.
    """
    def __init__(self, node: DQCNode):
        super().__init__(node)
        self.name = f"{self.node.name}.TeleportApp"
        node.teleport_app = self   # register ourselves so incoming TeleportMessage lands here:
        self.results = []          # where we’ll collect Bob’s teleported state
        self.teleport_protocols: list[TeleportProtocol] = [] # a list of teleport protocol instances
        log.logger.debug(f"{self.name}: initialized")

    def start(self, responder: str, start_t: int, end_t: int, memory_size: int, fidelity: float, data_memory_index: int):
        """Start the teleportation process.

        NOTE: only teleport one data memory qubit

        Args: 
            responder (str): Name of the responder node (Bob).
            start_t (int): Start time of the teleportation (in ps).
            end_t (int): End time of the teleportation (in ps).
            memory_size (int): Size of the memory used for the teleportation.
            fidelity (float): Target fidelity of the teleportation.
            data_memory_index (int): Index of the data qubit to be teleported.
        """
        log.logger.debug(f"{self.name}: start() → responder={responder}, data_memory_index={data_memory_index}")

        # reserve and generate EPR pair
        super().start(responder, start_t, end_t, memory_size, fidelity)

        # init a new teleportation protocol for Alice only, and append to the list
        teleport_protocol = TeleportProtocol(self.node, alice=True, data_memory_index=data_memory_index, remote_node_name=responder)
        self.teleport_protocols.append(teleport_protocol)


    def get_reservation_result(self, reservation, result: bool):
        """Handle the reservation result from the network manager.

        Args:
            reservation (Reservation): The reservation object.
            result (bool): True if the reservation was successful, False otherwise.
        """
        super().get_reservation_result(reservation, result)
        log.logger.debug(f"{self.name}: reservation_result → {result}")

    def get_memory(self, info: MemoryInfo):
        """Handle memory state changes.

        Args:
            info (MemoryInfo): Information about the memory state change.
        """
        log.logger.debug(f"{self.name}: get_memory, name={info.memory.name}, state={info.state}")
        # once we see our entangled half, hand it to the protocol
        if info.index in self.memo_to_reservation:
            if info.state == "ENTANGLED":
                for teleport_protocol in self.teleport_protocols:
                    this_node = info.memory.owner.name
                    remote_node = info.remote_node
                    if this_node == teleport_protocol.owner.name and remote_node == teleport_protocol.remote_node_name:
                        # this node is Alice
                        teleport_protocol.set_alice_comm_memory_name(info.memory.name)
                        teleport_protocol.set_alice_comm_memory(info.memory)
                        teleport_protocol.set_bob_comm_memory_name(info.remote_memo)
                        reservation = self.memo_to_reservation[info.index]
                        # Let Bob first execute EntanglementGenerationA._entanglement_succeed(), then let Alice do the Bell measurement
                        time_now = self.node.timeline.now()
                        process = Process(teleport_protocol, 'alice_bell_measurement', [reservation])
                        priority = self.node.timeline.schedule_counter
                        event = Event(time_now, process, priority)
                        self.node.timeline.schedule(event)
                        break # if never reached this break, then go to else
                else:
                    # this node is Bob, create the new teleport protocol instance, then append to self.teleport_protocols
                    teleport_protocol = TeleportProtocol(self.node, alice=False, remote_node_name=info.remote_node)
                    teleport_protocol.set_bob_comm_memory_name(info.memory.name)
                    teleport_protocol.set_bob_comm_memory(info.memory)
                    teleport_protocol.set_alice_comm_memory_name(info.remote_memo)
                    self.teleport_protocols.append(teleport_protocol)

    def received_message(self, src: str, msg: TeleportMessage):
        """Handle incoming teleport messages.

        Args:
            src (str): Source node name.
            msg (TeleportMessage): The teleport message received.
        """
        log.logger.debug(f"{self.name} received_message from {src}: {msg}")
        if msg.msg_type is TeleportMsgType.MEASUREMENT_RESULT:  # Bob receives measurement result from Alice
            for teleport_protocol in self.teleport_protocols:   # find the correct teleport protocol on Bob's side
                if src == teleport_protocol.remote_node_name and msg.bob_comm_memory_name == teleport_protocol.bob_comm_memory_name:
                    teleport_protocol.received_message(src, msg)
                    self.node.resource_manager.expire_rules_by_reservation(msg.reservation)                    # early release of resources
                    self.node.resource_manager.update(None, teleport_protocol.bob_comm_memory, MemoryInfo.RAW) # release the bob comm memory
                    teleport_protocol.bob_acknowledge_complete(msg.reservation)
                    self.teleport_protocols.remove(teleport_protocol)  # remove the protocol instance, it's lifecycle is complete
                    break
            else:
                log.logger.warning(f"{self.name}: received_message: no matching teleport protocol for msg={msg} from {src}")

        elif msg.msg_type is TeleportMsgType.ACK:              # Alice receives acknowledgment from Bob
            for teleport_protocol in self.teleport_protocols:  # find the correct teleport protocol on Alice's side
                if src == teleport_protocol.remote_node_name and msg.bob_comm_memory_name == teleport_protocol.bob_comm_memory_name:
                    self.node.resource_manager.expire_rules_by_reservation(msg.reservation)                      # expire the rules
                    self.node.resource_manager.update(None, teleport_protocol.alice_comm_memory, MemoryInfo.RAW) # release the alice comm memory
                    self.teleport_protocols.remove(teleport_protocol)  # remove the protocol instance, it's lifecycle is complete
                    break
            else:
                log.logger.warning(f"{self.name}: received_message: no matching teleport protocol for msg={msg} from {src}")

    def teleport_complete(self, comm_key: int):
        """Called by TeleportProtocol once Bob's qubit is corrected. comm_key holds the teleported |ψ⟩.

        Args:
            comm_key (int): The key of the comm memory where the teleported state is stored.
        """
        my_qubit = self.node.timeline.quantum_manager.get(comm_key)
        psi = my_qubit.state # get qubit state
        log.logger.info(f"{self.name}: teleport done, state={psi}")
        self.results.append((self.node.timeline.now(), psi)) # append result (timestamp, state)


### 2.2 The teleport() function

Teleport a $\ket{\psi}$ from Alice to Bob

In [3]:
def teleport(psi: np.array, seed: int = 0) -> np.array:
    """Do a teleportation
    
    Args:
        psi (np.array): the state to be teleported at Alice
        seed (int): the random seed
    Return:
        np.array: the state received at Bob
    """
    
    topo = DQCNetTopo("teleport_2node.json")
    tl   = topo.tl

    log_filename = 'teleport_2node.log'
    log.set_logger(__name__, tl, log_filename)
    log.set_logger_level('INFO')
    modules = ['generation', 'teleportation']
    for module in modules:
        log.track_module(module)

    nodes = topo.nodes[DQCNetTopo.DQC_NODE]
    alice = next(n for n in nodes if n.name=="alice")
    bob   = next(n for n in nodes if n.name=="bob")

    # set seed
    bsm_nodes = topo.nodes.get(DQCNetTopo.BSM_NODE, [])
    for bsm_node in bsm_nodes:
        bsm_node.set_seed(seed)
    
    # 1) Prepare |ψ> in Alice’s data qubit
    data_memo_arr = alice.get_component_by_name(alice.data_memo_arr_name)
    data_memo_arr[0].update_state(psi)

    # 2) Attach the TeleportApp on both nodes
    alice_app = TeleportApp(alice)
    bob_app   = TeleportApp(bob)

    # 3) Kick off teleport
    alice_app.start(
        responder   = bob.name,
        start_t     = 1  * MILLISECOND,
        end_t       = 100 * MILLISECOND,
        memory_size = 1,
        fidelity    = 0.8,
        data_memory_index = 0
    )
    
    # 4) Run the simulation
    tl.init()
    tl.run()
    
    # 5) Read out Bob’s communication qubit state
    timestamp, teleported_qubit = bob_app.results[0]

    return np.array(teleported_qubit)

### 2.3 The main() function

In [4]:
def main():

    psi = np.array(random_state())
    
    seed = np.random.randint(0, 1000)
    teleported_psi = teleport(psi, seed)  # run the teleporation
    
    print(f'psi            = {psi}')
    print(f'teleported psi = {teleported_psi}')
    
    assert verify_same_state_vector(teleported_psi, psi), f"teleported state {teleported_psi} != original {psi}"
    
    print('\npsi = teleported_psi --> teleporation success!')
    
    print('\nThe teleport_2node.log:\n')
    with open("teleport_2node.log", "r") as f:
        logs = f.read()
        print(logs)

In [5]:
# please run this cell repeatedly
main()

psi            = [0.80879701+0.j         0.24967705+0.53245541j]
teleported psi = [0.80879701+0.j         0.24967705+0.53245541j]

psi = teleported_psi --> teleporation success!

The teleport_2node.log:

4,505,012,510        INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
4,505,012,510        INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
6,507,525,010        INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
6,507,525,010        INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
10,012,550,010       INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
10,012,550,010       INFO    generation             bob failed entanglement of memory bob.MemoryArray[0]
12,015,062,510       INFO    generation             alice failed entanglement of memory alice.MemoryArray[0]
12,015,062,510       INFO    generation      